# Notebook for cleaning `ORGANIZATION TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
select('organizations')

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
0,ef58ea08-d883-3957-8300-150554edc8fb,HEALTHALLIANCE HOSPITALS INC,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876,9784662000,198002.28,1557
1,69176529-fd1f-3b3f-abce-a0a3626769eb,MOUNT AUBURN HOSPITAL,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6174923500,288569.17,2296
2,5e765f2b-e908-3888-9fc7-df2cb87beb58,STURDY MEMORIAL HOSPITAL,211 PARK STREET,ATTLEBORO,MA,02703,41.931653,-71.294503,5082225200,287355.03,2287
3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,LAWRENCE GENERAL HOSPITAL,ONE GENERAL STREET,LAWRENCE,MA,01842,42.700273,-71.161357,9786834000,163386.98,1327
4,e002090d-4e92-300e-b41e-7d1f21dee4c6,CAMBRIDGE HEALTH ALLIANCE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6176652300,406053.19,3199


In [3]:
query = '''
    alter table organizations
    drop column address,
    drop column zip,
    drop column lat,
    drop column lon,
    drop column phone;
'''
run(query)

In [4]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'organizations' and
    table_schema = database();
'''
sql(query)
organizatios_columns = [c[0] for c in sql(query).values.tolist()]

In [5]:
for col in organizatios_columns:
    query = f'''
        alter table organizations
        rename column {col} to {col.lower()}
    '''
    run(query)

In [6]:
for col in organizatios_columns:
    query = f'''
        update organizations
        set {col} = lower({col})
    '''
    run(query)

In [7]:
select('organizations')

,id,name,city,state,revenue,utilization
0,ef58ea08-d883-3957-8300-150554edc8fb,healthalliance hospitals inc,leominster,ma,198002.28,1557
1,69176529-fd1f-3b3f-abce-a0a3626769eb,mount auburn hospital,cambridge,ma,288569.17,2296
2,5e765f2b-e908-3888-9fc7-df2cb87beb58,sturdy memorial hospital,attleboro,ma,287355.03,2287
3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,lawrence general hospital,lawrence,ma,163386.98,1327
4,e002090d-4e92-300e-b41e-7d1f21dee4c6,cambridge health alliance,cambridge,ma,406053.19,3199
